In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

### Load the data

In [3]:
data = pd.read_csv("../data/alzheimers_disease_data.csv")
data = data.drop("DoctorInCharge", axis=1) # this attribute is confidential in the data, and thus not useful 
data.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
0,4751,73,0,0,2,22.927749,0,13.297218,6.327112,1.347214,...,6.518877,0,0,1.725883,0,0,0,1,0,0
1,4752,89,0,0,0,26.827681,0,4.542524,7.619885,0.518767,...,7.118696,0,0,2.592424,0,0,0,0,1,0
2,4753,73,0,3,1,17.795882,0,19.555085,7.844988,1.826335,...,5.895077,0,0,7.119548,0,1,0,1,0,0
3,4754,74,1,0,1,33.800817,1,12.209266,8.428001,7.435604,...,8.965106,0,1,6.481226,0,0,0,0,0,0
4,4755,89,0,0,0,20.716974,0,18.454356,6.310461,0.795498,...,6.045039,0,0,0.014691,0,0,1,1,0,0


### Split the data

In [6]:
X, y = data, data["Diagnosis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=22, stratify=y)
X_test = X_test.drop("Diagnosis", axis=1)

X_train.head()

,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,AlcoholConsumption,PhysicalActivity,DietQuality,...,FunctionalAssessment,MemoryComplaints,BehavioralProblems,ADL,Confusion,Disorientation,PersonalityChanges,DifficultyCompletingTasks,Forgetfulness,Diagnosis
1062,5813,74,0,0,2,16.327315,0,11.281357,1.151913,7.171360,...,0.921736,0,1,4.217614,1,0,0,0,0,1
1982,6733,71,0,1,1,15.138279,0,3.570242,0.331743,3.376891,...,5.492227,0,1,5.023993,0,0,0,0,1,1
1390,6141,88,1,0,2,35.751527,0,10.876879,7.052473,9.314380,...,4.266609,0,1,9.352727,0,1,0,1,0,1
243,4994,70,0,0,2,23.185997,0,8.451036,6.184948,9.927077,...,4.225134,0,0,5.454608,1,0,0,0,1,0
76,4827,68,1,0,1,24.487589,0,14.060047,8.112291,9.864426,...,5.815445,0,0,3.301422,1,0,0,0,1,0


In [9]:
X_temp = X_train.drop("Ethnicity", axis=1)
y_temp = X_train["Ethnicity"]

smote = SMOTE(random_state=12)
X_resampled_temp, y_resampled_temp = smote.fit_resample(X_temp, y_temp)

X_oversampled_temp = pd.DataFrame(X_resampled_temp, columns=X_temp.columns)
X_oversampled_temp.insert(loc=3, column="Ethnicity", value=None)
X_oversampled_temp["Ethnicity"] = y_resampled_temp

X_train_smote = X_oversampled_temp.drop("Diagnosis", axis=1)
y_train_smote = X_oversampled_temp["Diagnosis"]

print('Oversampled dataset shape: %s' % X_train_smote["Ethnicity"].value_counts())

Oversampled dataset shape: Ethnicity
0    932
1    932
3    932
2    932
Name: count, dtype: int64


### Train HistGradientBoostingClassifier

In [10]:
# class_weight is **NOT** balanced anymore!! Because thatt's done during the SMOTE synthesis
classifier = HistGradientBoostingClassifier(random_state=22)
classifier.fit(X_train_smote, y_train_smote)

y_pred = classifier.predict(X_test)

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95       348
           1       0.95      0.87      0.91       190

    accuracy                           0.94       538
   macro avg       0.94      0.92      0.93       538
weighted avg       0.94      0.94      0.94       538



### Compare performance based on ethnicity

In [12]:
# Join X_test, y_test, y_pred for analysis
results = X_test.copy()
results["TrueDiagnosis"] = y_test
results["PredictedDiagnosis"] = y_pred

In [13]:
# Ethnicity 0
eth0_results = results[results["Ethnicity"] == 0]
print("Ethnicity 0 Results:")
print(classification_report(eth0_results["TrueDiagnosis"], eth0_results["PredictedDiagnosis"]))

Ethnicity 0 Results:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       222
           1       0.97      0.92      0.95       124

    accuracy                           0.96       346
   macro avg       0.97      0.95      0.96       346
weighted avg       0.96      0.96      0.96       346



In [14]:
# Ethnicity 1
eth1_results = results[results["Ethnicity"] == 1]
print("Ethnicity 1 Results:")
print(classification_report(eth1_results["TrueDiagnosis"], eth1_results["PredictedDiagnosis"]))

Ethnicity 1 Results:
              precision    recall  f1-score   support

           0       0.87      0.94      0.90        63
           1       0.85      0.72      0.78        32

    accuracy                           0.86        95
   macro avg       0.86      0.83      0.84        95
weighted avg       0.86      0.86      0.86        95



In [15]:
# Ethnicity 2
eth2_results = results[results["Ethnicity"] == 2]
print("Ethnicity 2 Results:")
print(classification_report(eth2_results["TrueDiagnosis"], eth2_results["PredictedDiagnosis"]))

Ethnicity 2 Results:
              precision    recall  f1-score   support

           0       0.81      0.96      0.88        23
           1       0.94      0.75      0.83        20

    accuracy                           0.86        43
   macro avg       0.88      0.85      0.86        43
weighted avg       0.87      0.86      0.86        43



In [16]:
# Ethnicity 3
eth3_results = results[results["Ethnicity"] == 3]
print("Ethnicity 3 Results:")
print(classification_report(eth3_results["TrueDiagnosis"], eth3_results["PredictedDiagnosis"]))

Ethnicity 3 Results:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        40
           1       1.00      0.93      0.96        14

    accuracy                           0.98        54
   macro avg       0.99      0.96      0.98        54
weighted avg       0.98      0.98      0.98        54



In [21]:
%load_ext autoreload
%autoreload 2
from fairness_metrics import demographic_parity, equalized_odds, disparate_impact

# DI >= 0.8 is a pre-established threshold for fairness
# DP and EO need to be as close to 0 as possible

for i in [1,2,3]:
    dp = round(demographic_parity(results, 0, i), 4)
    eo = equalized_odds(results, 0, i)
    tpr_diff, fpr_diff = round(eo[0], 4), round(eo[1], 4)
    di = round(disparate_impact(results, 0, i), 4)
    print(dp, tpr_diff, fpr_diff, di)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.0539 -0.2006 0.05 1.1898
0.0339 -0.1694 0.03 0.9088
0.0974 0.0092 -0.0135 1.4046
